In [532]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, LSTM
from sklearn.preprocessing import MinMaxScaler
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
import time

In [533]:
df=pd.read_csv('COVID-19 data for GCC countries.csv', index_col='date', parse_dates=True)
df

,Bahrain_new_cases,Bahrain_Mean_Tem,Bahrain_Mean_Humidity,Bahrain_Mean_Dewpoint,Bahrain_Mean_Windspeed,Qatar_new_cases,Qatar_Mean_Temp,Qatar_Mean_Humidity,Qatar_Mean_Dewpoint,Qatar_Mean_Windspeed,...,Oman_new_cases,Oman_Mean_Temp,Oman_Mean_Humidity,Oman_Mean_Dewpoint,Oman_Mean_Windspeed,UAE_new_cases,UAE_Mean_Temp,UAE_Mean_Humidity,UAE_Mean_Dewpoint,UAE_Mean_Windspeed
date,,,,,,,,,,,,,,,,,,,,,
2020-04-17,23,25,66,53,6.0,392,25,64,62,5,...,50,27,80,73,6,460,25,69,66,8
2020-04-18,44,28,62,54,9.0,560,25,67,65,8,...,111,27,74,72,7,477,26,67,67,8
2020-04-19,29,25,70,64,9.0,345,26,77,71,8,...,86,28,79,76,7,0,30,42,58,7
2020-04-20,108,25,72,60,6.0,440,27,65,67,8,...,144,30,67,73,7,479,31,39,59,9
2020-04-21,26,26,67,57,6.0,567,27,59,64,7,...,98,31,47,64,8,484,29,47,62,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-07,105,59,53,53,5.0,189,35,47,72,5,...,80,32,77,81,5,977,34,66,80,7
2021-09-08,139,46,66,66,7.0,172,35,61,79,6,...,67,33,76,81,6,952,36,58,77,7
2021-09-09,114,31,51,51,17.0,0,37,50,75,11,...,52,32,70,79,5,833,36,61,80,9


In [534]:
df.isnull().sum()

Bahrain_new_cases         0
Bahrain_Mean_Tem          0
Bahrain_Mean_Humidity     0
Bahrain_Mean_Dewpoint     0
Bahrain_Mean_Windspeed    0
Qatar_new_cases           0
Qatar_Mean_Temp           0
Qatar_Mean_Humidity       0
Qatar_Mean_Dewpoint       0
Qatar_Mean_Windspeed      0
KSA_new_cases             0
KSA_Mean_Temp             0
KSA_Mean_Humidity         0
KSA_Mean_Dewpoint         0
KSA_Mean_Windspeed        0
Kuwait_new_cases          0
Kuwait_Mean_Temp          0
Kuwait_Mean_Humidity      0
Kuwait_Mean_Dewpoint      0
Kuwait_Mean_Windspeed     0
Oman_new_cases            0
Oman_Mean_Temp            0
Oman_Mean_Humidity        0
Oman_Mean_Dewpoint        0
Oman_Mean_Windspeed       0
UAE_new_cases             0
UAE_Mean_Temp             0
UAE_Mean_Humidity         0
UAE_Mean_Dewpoint         0
UAE_Mean_Windspeed        0
dtype: int64

In [750]:
UAECOVID=df[["UAE_new_cases","UAE_Mean_Temp"]]
UAECOVID.shape

(513, 2)

In [751]:
scalarUAECOVID=MinMaxScaler()
scalarUAECOVID.fit(UAECOVID)
normalizedUAECOVID=scalarUAECOVID.transform(UAECOVID)

print(type(normalizedUAECOVID[2][0]))
print(normalizedUAECOVID[0][0],normalizedUAECOVID[0][1])

<class 'numpy.float64'>
0.11566507417651496 0.30434782608695654


In [752]:
from numpy import hstack
n_input=7
n_output=1
casesUAE=[]
UAEtemp=[]

number=normalizedUAECOVID.shape[0]
print(number)
print(type(normalizedUAECOVID))
targetUAE=[]
targetCases=normalizedUAECOVID[:,0]
for i in range(0,number):
    casesUAE.append(normalizedUAECOVID[i][0])
    UAEtemp.append(normalizedUAECOVID[i][1])
    
casesUAE=np.array(casesUAE)
UAEtemp=np.array(UAEtemp)
print(len(casesUAE),len(UAEtemp))
casesUAE=casesUAE.reshape(len(casesUAE),1)
UAEtemp=UAEtemp.reshape(len(UAEtemp),1)
print(casesUAE.shape)
datasetUAE=hstack((casesUAE,UAEtemp))
print(datasetUAE.shape)
generatorUAE=TimeseriesGenerator(datasetUAE,targetCases,length=n_input,batch_size=1)
print(generatorUAE[1])

513
<class 'numpy.ndarray'>
513 513
(513, 1)
(513, 2)
(array([[[0.11993965, 0.34782609],
        [0.        , 0.52173913],
        [0.12044254, 0.56521739],
        [0.12169977, 0.47826087],
        [0.12320845, 0.47826087],
        [0.12144833, 0.47826087],
        [0.13024893, 0.56521739]]]), array([0.13200905]))


In [753]:
dataUAE=[]
targetUAE=[]
for i in range(len(generatorUAE)):
    data1,target1=generatorUAE[i]
    dataUAE.append(data1[0,:,:])
    xx=float(target1)
    targetUAE.append(xx)
dataUAE=np.array(dataUAE)
targetUAE=np.array(targetUAE)
print(type(dataUAE))
print(dataUAE.shape)
print(targetUAE.shape)
x_train_UAE,x_test_UAE,y_train_UAE,y_test_UAE=train_test_split(dataUAE,targetUAE,test_size=0.2,random_state=4)
print(type(dataUAE))
print(x_train_UAE.shape)

<class 'numpy.ndarray'>
(506, 7, 2)
(506,)
<class 'numpy.ndarray'>
(404, 7, 2)


In [754]:
def build_model(hp):
    model=keras.models.Sequential()
    model.add(LSTM((hp.Int("input_units",min_value=10,max_value=300,step=10)),input_shape=(x_train_UAE.shape[1],x_train_UAE.shape[2]),activation='sigmoid',return_sequences=True))
    for i in range(hp.Int("n_layers",1,4)):
        model.add(LSTM(hp.Int(f"lstm_{i}_units",min_value=10,max_value=300,step=10),activation='sigmoid',return_sequences=True))
        
    model.add(LSTM((hp.Int("output_units",min_value=1,max_value=1,step=1)),activation='tanh',return_sequences=False))
    model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    return model

In [755]:
UAE_bivar=f"{int(time.time())}"

In [756]:
tuner =RandomSearch(
    build_model,
    objective="loss",
    overwrite=False,
    max_trials=3,
    executions_per_trial=1,
    directory=UAE_bivar)

In [757]:
tuner.search(x=x_train_UAE, y=y_train_UAE, epochs=40,batch_size=1,validation_data=(x_test_UAE,y_test_UAE))

Trial 3 Complete [00h 02m 04s]
loss: 0.002285297028720379

Best loss So Far: 0.0022587452549487352
Total elapsed time: 00h 05m 20s
INFO:tensorflow:Oracle triggered exit


In [758]:
import pickle 

In [759]:
print(tuner.get_best_hyperparameters()[0].values)

{'input_units': 170, 'n_layers': 1, 'lstm_0_units': 80, 'output_units': 1}


In [760]:
print(tuner.results_summary())

Results summary
Results in 1639093083\untitled_project
Showing 10 best trials
Objective(name='loss', direction='min')
Trial summary
Hyperparameters:
input_units: 170
n_layers: 1
lstm_0_units: 80
output_units: 1
Score: 0.0022587452549487352
Trial summary
Hyperparameters:
input_units: 190
n_layers: 2
lstm_0_units: 70
output_units: 1
lstm_1_units: 10
Score: 0.002285297028720379
Trial summary
Hyperparameters:
input_units: 30
n_layers: 1
lstm_0_units: 120
output_units: 1
Score: 0.002358224242925644
None


In [761]:
print(tuner.get_best_models()[0].summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 170)            117640    
                                                                 
 lstm_1 (LSTM)               (None, 7, 80)             80320     
                                                                 
 lstm_2 (LSTM)               (None, 1)                 328       
                                                                 
Total params: 198,288
Trainable params: 198,288
Non-trainable params: 0
_________________________________________________________________
None


In [762]:
KSACOVID=df[["KSA_new_cases","KSA_Mean_Temp"]]
KSACOVID.shape

(513, 2)

In [763]:
scalarKSACOVID=MinMaxScaler()
scalarKSACOVID.fit(KSACOVID)
normalizedKSACOVID=scalarKSACOVID.transform(KSACOVID)

print(type(normalizedKSACOVID[2][0]))
print(normalizedKSACOVID[0][0],normalizedKSACOVID[0][1])

<class 'numpy.float64'>
0.09013851560884845 0.375


In [764]:
from numpy import hstack
n_input=7
n_output=1
casesKSA=[]
KSAtemp=[]
number=normalizedKSACOVID.shape[0]
print(number)
print(type(normalizedKSACOVID))
targetKSA=[]
targetCases1=normalizedKSACOVID[:,0]
for i in range(0,number):
    casesKSA.append(normalizedKSACOVID[i][0])
    KSAtemp.append(normalizedKSACOVID[i][1])
    
casesKSA=np.array(casesKSA)
KSAtemp=np.array(KSAtemp)

print(len(casesKSA),len(KSAtemp))
casesKSA=casesKSA.reshape(len(casesKSA),1)
KSAtemp=KSAtemp.reshape(len(KSAtemp),1)

print(casesKSA.shape)
datasetKSA=hstack((casesKSA,KSAtemp))
print(datasetKSA.shape)
generatorKSA=TimeseriesGenerator(datasetKSA,targetCases1,length=n_input,batch_size=1)
print(generatorKSA[1])

513
<class 'numpy.ndarray'>
513 513
(513, 1)
(513, 2)
(array([[[0.14058301, 0.375     ],
        [0.2170767 , 0.4375    ],
        [0.20798015, 0.5       ],
        [0.2150093 , 0.4375    ],
        [0.2201778 , 0.375     ],
        [0.21893736, 0.5       ],
        [0.22245193, 0.5625    ]]]), array([0.22534629]))


In [765]:
dataKSA=[]
targetKSA=[]
for i in range(len(generatorKSA)):
    data2,target2=generatorKSA[i]
    dataKSA.append(data2[0,:,:])
    xx=float(target2)
    targetKSA.append(xx)
dataKSA=np.array(dataKSA)
targetKSA=np.array(targetKSA)
print(type(dataKSA))
print(dataKSA.shape)
print(targetKSA.shape)
x_train_KSA,x_test_KSA,y_train_KSA,y_test_KSA=train_test_split(dataKSA,targetKSA,test_size=0.2,random_state=4)
print(type(dataKSA))
print(x_train_KSA.shape)

<class 'numpy.ndarray'>
(506, 7, 2)
(506,)
<class 'numpy.ndarray'>
(404, 7, 2)


In [766]:
def build_model1(hp):
    model1=keras.models.Sequential()
    model1.add(LSTM((hp.Int("input_units",min_value=10,max_value=300,step=10)),input_shape=(x_train_KSA.shape[1],x_train_KSA.shape[2]),activation='sigmoid',return_sequences=True))
    
    for i in range(hp.Int("n_layers",1,4)):
        model1.add(LSTM(hp.Int(f"lstm_{i}_units",min_value=10,max_value=300,step=10),activation='sigmoid',return_sequences=True))
        
    model1.add(LSTM((hp.Int("output_units",min_value=1,max_value=1,step=1)),activation='tanh',return_sequences=False))
    model1.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    return model1

In [767]:
KSA_bivar=f"{int(time.time())}"

In [768]:
tuner =RandomSearch(
    build_model1,
    objective="loss",
    overwrite=False,
    max_trials=3,
    executions_per_trial=1,
    directory=KSA_bivar)

In [769]:
tuner.search(x=x_train_KSA, y=y_train_KSA, epochs=40,batch_size=1,validation_data=(x_test_KSA,y_test_KSA))

Trial 3 Complete [00h 03m 09s]
loss: 0.0011855403427034616

Best loss So Far: 0.001128960051573813
Total elapsed time: 00h 07m 38s
INFO:tensorflow:Oracle triggered exit


In [770]:
print(tuner.get_best_hyperparameters()[0].values)

{'input_units': 110, 'n_layers': 1, 'lstm_0_units': 120, 'output_units': 1}


In [771]:
print(tuner.results_summary())

Results summary
Results in 1639093582\untitled_project
Showing 10 best trials
Objective(name='loss', direction='min')
Trial summary
Hyperparameters:
input_units: 110
n_layers: 1
lstm_0_units: 120
output_units: 1
Score: 0.001128960051573813
Trial summary
Hyperparameters:
input_units: 60
n_layers: 2
lstm_0_units: 140
output_units: 1
lstm_1_units: 280
lstm_2_units: 240
Score: 0.0011855403427034616
Trial summary
Hyperparameters:
input_units: 190
n_layers: 3
lstm_0_units: 170
output_units: 1
lstm_1_units: 10
lstm_2_units: 10
Score: 0.0015269230352714658
None


In [772]:
print(tuner.get_best_models()[0].summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 110)            49720     
                                                                 
 lstm_1 (LSTM)               (None, 7, 120)            110880    
                                                                 
 lstm_2 (LSTM)               (None, 1)                 488       
                                                                 
Total params: 161,088
Trainable params: 161,088
Non-trainable params: 0
_________________________________________________________________
None


In [773]:
KuwaitCOVID=df[["Kuwait_new_cases","Kuwait_Mean_Temp"]]
KuwaitCOVID.shape

(513, 2)

In [774]:
scalarKuwaitCOVID=MinMaxScaler()
scalarKuwaitCOVID.fit(KuwaitCOVID)
normalizedKuwaitCOVID=scalarKuwaitCOVID.transform(KuwaitCOVID)

print(type(normalizedKuwaitCOVID[2][0]))
print(normalizedKuwaitCOVID[0][0],normalizedKuwaitCOVID[0][1])

<class 'numpy.float64'>
0.059708981435022575 0.4705882352941176


In [775]:
from numpy import hstack
n_input=7
n_output=1
casesKuwait=[]
Kuwaittemp=[]

number=normalizedKuwaitCOVID.shape[0]
print(number)
print(type(normalizedKuwaitCOVID))
targetKuwait=[]
targetCases3=normalizedKuwaitCOVID[:,0]
for i in range(0,number):
    casesKuwait.append(normalizedKuwaitCOVID[i][0])
    Kuwaittemp.append(normalizedKuwaitCOVID[i][1])

casesKuwait=np.array(casesKuwait)
Kuwaittemp=np.array(Kuwaittemp)

print(len(casesKuwait),len(Kuwaittemp))
casesKuwait=casesKuwait.reshape(len(casesKuwait),1)
Kuwaittemp=Kuwaittemp.reshape(len(Kuwaittemp),1)

print(casesKuwait.shape)
datasetKuwait=hstack((casesKuwait,Kuwaittemp))
print(datasetKuwait.shape)
generatorKuwait=TimeseriesGenerator(datasetKuwait,targetCases3,length=n_input,batch_size=1)
print(generatorKuwait[1])

513
<class 'numpy.ndarray'>
513 513
(513, 1)
(513, 2)
(array([[[0.06723532, 0.41176471],
        [0.04666332, 0.47058824],
        [0.08228801, 0.5       ],
        [0.04014049, 0.44117647],
        [0.04264927, 0.44117647],
        [0.08429503, 0.47058824],
        [0.07576518, 0.55882353]]]), array([0.10787757]))


In [776]:
dataKuwait=[]
targetKuwait=[]
for i in range(len(generatorKuwait)):
    data4,target4=generatorKuwait[i]
    dataKuwait.append(data4[0,:,:])
    xx=float(target4)
    targetKuwait.append(xx)
dataKuwait=np.array(dataKuwait)
targetKuwait=np.array(targetKuwait)
print(type(dataKuwait))
print(dataKuwait.shape)
print(targetKuwait.shape)
x_train_Kuwait,x_test_Kuwait,y_train_Kuwait,y_test_Kuwait=train_test_split(dataKuwait,targetKuwait,test_size=0.2,random_state=4)
print(type(dataKuwait))
print(x_train_Kuwait.shape)

<class 'numpy.ndarray'>
(506, 7, 2)
(506,)
<class 'numpy.ndarray'>
(404, 7, 2)


In [777]:
def build_model2(hp):
    model2=keras.models.Sequential()
    model2.add(LSTM((hp.Int("input_units",min_value=10,max_value=300,step=10)),input_shape=(x_train_Kuwait.shape[1],x_train_Kuwait.shape[2]),activation='sigmoid',return_sequences=True))
    for i in range(hp.Int("n_layers",1,4)):
        model2.add(LSTM(hp.Int(f"lstm_{i}_units",min_value=10,max_value=300,step=10),activation='sigmoid',return_sequences=True))
        
    model2.add(LSTM((hp.Int("output_units",min_value=1,max_value=1,step=1)),activation='tanh',return_sequences=False))
    model2.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    return model2

In [778]:
Kuwait_bivar=f"{int(time.time())}"

In [779]:
tuner =RandomSearch(
    build_model2,
    objective="loss",
    overwrite=False,
    max_trials=3,
    executions_per_trial=1,
    directory=Kuwait_bivar)

In [780]:
tuner.search(x=x_train_Kuwait, y=y_train_Kuwait, epochs=40,batch_size=1,validation_data=(x_test_Kuwait,y_test_Kuwait))

Trial 3 Complete [00h 02m 58s]
loss: 0.0047593326307833195

Best loss So Far: 0.0047593326307833195
Total elapsed time: 00h 09m 54s
INFO:tensorflow:Oracle triggered exit


In [781]:
print(tuner.get_best_hyperparameters()[0].values)

{'input_units': 60, 'n_layers': 3, 'lstm_0_units': 50, 'output_units': 1, 'lstm_1_units': 140, 'lstm_2_units': 200, 'lstm_3_units': 80}


In [782]:
print(tuner.results_summary())

Results summary
Results in 1639098182\untitled_project
Showing 10 best trials
Objective(name='loss', direction='min')
Trial summary
Hyperparameters:
input_units: 60
n_layers: 3
lstm_0_units: 50
output_units: 1
lstm_1_units: 140
lstm_2_units: 200
lstm_3_units: 80
Score: 0.0047593326307833195
Trial summary
Hyperparameters:
input_units: 300
n_layers: 3
lstm_0_units: 30
output_units: 1
lstm_1_units: 10
lstm_2_units: 10
Score: 0.005388186313211918
Trial summary
Hyperparameters:
input_units: 210
n_layers: 4
lstm_0_units: 190
output_units: 1
lstm_1_units: 40
lstm_2_units: 260
lstm_3_units: 10
Score: 0.012061669491231441
None


In [783]:
print(tuner.get_best_models()[0].summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 60)             15120     
                                                                 
 lstm_1 (LSTM)               (None, 7, 50)             22200     
                                                                 
 lstm_2 (LSTM)               (None, 7, 140)            106960    
                                                                 
 lstm_3 (LSTM)               (None, 7, 200)            272800    
                                                                 
 lstm_4 (LSTM)               (None, 1)                 808       
                                                                 
Total params: 417,888
Trainable params: 417,888
Non-trainable params: 0
_________________________________________________________________
None


In [784]:
BahrainCOVID=df[["Bahrain_new_cases","Bahrain_Mean_Tem"]]
BahrainCOVID.shape

(513, 2)

In [785]:
scalarBahrainCOVID=MinMaxScaler()
scalarBahrainCOVID.fit(BahrainCOVID)
normalizedBahrainCOVID=scalarBahrainCOVID.transform(BahrainCOVID)

print(type(normalizedBahrainCOVID[2][0]))
print(normalizedBahrainCOVID[0][0],normalizedBahrainCOVID[0][1])

<class 'numpy.float64'>
0.0 0.2272727272727273


In [786]:
from numpy import hstack
n_input=7
n_output=1
casesBahrain=[]
Bahraintemp=[]

number=normalizedBahrainCOVID.shape[0]
print(number)
print(type(normalizedBahrainCOVID))
targetBahrain=[]
targetCases2=normalizedBahrainCOVID[:,0]
for i in range(0,number):
    casesBahrain.append(normalizedBahrainCOVID[i][0])
    Bahraintemp.append(normalizedBahrainCOVID[i][1])

casesBahrain=np.array(casesBahrain)
Bahraintemp=np.array(Bahraintemp)

print(len(casesBahrain),len(Bahraintemp))
casesBahrain=casesBahrain.reshape(len(casesBahrain),1)
Bahraintemp=Bahraintemp.reshape(len(Bahraintemp),1)

print(casesBahrain.shape)
datasetBahrain=hstack((casesBahrain,Bahraintemp))
print(datasetBahrain.shape)
generatorBahrain=TimeseriesGenerator(datasetBahrain,targetCases2,length=n_input,batch_size=1)
print(generatorBahrain[1])

513
<class 'numpy.ndarray'>
513 513
(513, 1)
(513, 2)
(array([[[0.00645756, 0.29545455],
        [0.00184502, 0.22727273],
        [0.02613776, 0.22727273],
        [0.00092251, 0.25      ],
        [0.01322263, 0.25      ],
        [0.0095326 , 0.27272727],
        [0.05135301, 0.29545455]]]), array([0.08548585]))


In [787]:
dataBahrain=[]
targetBahrain=[]
for i in range(len(generatorBahrain)):
    data3,target3=generatorBahrain[i]
    dataBahrain.append(data3[0,:,:])
    xx=float(target3)
    targetBahrain.append(xx)
dataBahrain=np.array(dataBahrain)
targetBahrain=np.array(targetBahrain)
print(type(dataBahrain))
print(dataBahrain.shape)
print(targetBahrain.shape)
x_train_Bahrain,x_test_Bahrain,y_train_Bahrain,y_test_Bahrain=train_test_split(dataBahrain,targetBahrain,test_size=0.2,random_state=4)
print(type(dataBahrain))
print(x_train_Bahrain.shape)

<class 'numpy.ndarray'>
(506, 7, 2)
(506,)
<class 'numpy.ndarray'>
(404, 7, 2)


In [788]:
def build_model3(hp):
    model3=keras.models.Sequential()
    model3.add(LSTM((hp.Int("input_units",min_value=10,max_value=300,step=10)),input_shape=(x_train_Bahrain.shape[1],x_train_Bahrain.shape[2]),activation='sigmoid',return_sequences=True))
    for i in range(hp.Int("n_layers",1,4)):
        model3.add(LSTM(hp.Int(f"lstm_{i}_units",min_value=10,max_value=300,step=10),activation='sigmoid',return_sequences=True))
        
    model3.add(LSTM((hp.Int("output_units",min_value=1,max_value=1,step=1)),activation='tanh',return_sequences=False))
    model3.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    return model3

In [789]:
Bahrain_bivar=f"{int(time.time())}"

In [790]:
tuner =RandomSearch(
    build_model3,
    objective="loss",
    overwrite=False,
    max_trials=3,
    executions_per_trial=1,
    directory=Bahrain_bivar)

In [791]:
tuner.search(x=x_train_Bahrain, y=y_train_Bahrain, epochs=40,batch_size=1,validation_data=(x_test_Bahrain,y_test_Bahrain))

Trial 3 Complete [00h 08m 55s]
loss: 0.02728627622127533

Best loss So Far: 0.0014128845650702715
Total elapsed time: 00h 24m 24s
INFO:tensorflow:Oracle triggered exit


In [723]:
print(tuner.get_best_hyperparameters()[0].values)

{'input_units': 40, 'n_layers': 2, 'lstm_0_units': 230, 'output_units': 1, 'lstm_1_units': 180}


In [792]:
print(tuner.results_summary())

Results summary
Results in 1639098946\untitled_project
Showing 10 best trials
Objective(name='loss', direction='min')
Trial summary
Hyperparameters:
input_units: 110
n_layers: 2
lstm_0_units: 30
output_units: 1
lstm_1_units: 10
Score: 0.0014128845650702715
Trial summary
Hyperparameters:
input_units: 40
n_layers: 4
lstm_0_units: 30
output_units: 1
lstm_1_units: 270
lstm_2_units: 220
lstm_3_units: 30
Score: 0.02728627622127533
Trial summary
Hyperparameters:
input_units: 240
n_layers: 4
lstm_0_units: 230
output_units: 1
lstm_1_units: 270
lstm_2_units: 10
lstm_3_units: 10
Score: 0.02737380936741829
None


In [793]:
print(tuner.get_best_models()[0].summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 110)            49720     
                                                                 
 lstm_1 (LSTM)               (None, 7, 30)             16920     
                                                                 
 lstm_2 (LSTM)               (None, 7, 10)             1640      
                                                                 
 lstm_3 (LSTM)               (None, 1)                 48        
                                                                 
Total params: 68,328
Trainable params: 68,328
Non-trainable params: 0
_________________________________________________________________
None


In [794]:
QatarCOVID=df[["Qatar_new_cases","Qatar_Mean_Temp"]]
QatarCOVID.shape

(513, 2)

In [795]:
scalarQatarCOVID=MinMaxScaler()
scalarQatarCOVID.fit(QatarCOVID)
normalizedQatarCOVID=scalarQatarCOVID.transform(QatarCOVID)

print(type(normalizedQatarCOVID[2][0]))
print(normalizedQatarCOVID[0][0],normalizedQatarCOVID[0][1])

<class 'numpy.float64'>
0.1664543524416136 0.34615384615384615


In [796]:
from numpy import hstack
n_input=7
n_output=1
casesQatar=[]
Qatartemp=[]

number=normalizedQatarCOVID.shape[0]
print(number)
print(type(normalizedQatarCOVID))
targetQatar=[]
targetCases4=normalizedQatarCOVID[:,0]
for i in range(0,number):
    casesQatar.append(normalizedQatarCOVID[i][0])
    Qatartemp.append(normalizedQatarCOVID[i][1])

casesQatar=np.array(casesQatar)
Qatartemp=np.array(Qatartemp)

print(len(casesQatar),len(Qatartemp))
casesQatar=casesQatar.reshape(len(casesQatar),1)
Qatartemp=Qatartemp.reshape(len(Qatartemp),1)

print(casesQatar.shape)
datasetQatar=hstack((casesQatar,Qatartemp))
print(datasetQatar.shape)
generatorQatar=TimeseriesGenerator(datasetQatar,targetCases4,length=n_input,batch_size=1)
print(generatorQatar[1])

513
<class 'numpy.ndarray'>
513 513
(513, 1)
(513, 2)
(array([[[0.23779193, 0.34615385],
        [0.14649682, 0.38461538],
        [0.18683652, 0.42307692],
        [0.24076433, 0.42307692],
        [0.21995754, 0.42307692],
        [0.2581741 , 0.42307692],
        [0.26454352, 0.53846154]]]), array([0.32314225]))


In [797]:
dataQatar=[]
targetQatar=[]
for i in range(len(generatorQatar)):
    data5,target5=generatorQatar[i]
    dataQatar.append(data5[0,:,:])
    xx=float(target5)
    targetQatar.append(xx)
dataQatar=np.array(dataQatar)
targetQatar=np.array(targetQatar)
print(type(dataQatar))
print(dataQatar.shape)
print(targetQatar.shape)
x_train_Qatar,x_test_Qatar,y_train_Qatar,y_test_Qatar=train_test_split(dataQatar,targetQatar,test_size=0.2,random_state=4)
print(type(dataQatar))
print(x_train_Qatar.shape)

<class 'numpy.ndarray'>
(506, 7, 2)
(506,)
<class 'numpy.ndarray'>
(404, 7, 2)


In [798]:
def build_model4(hp):
    model4=keras.models.Sequential()
    model4.add(LSTM((hp.Int("input_units",min_value=10,max_value=300,step=10)),input_shape=(x_train_Qatar.shape[1],x_train_Qatar.shape[2]),activation='sigmoid',return_sequences=True))
    for i in range(hp.Int("n_layers",1,4)):
        model4.add(LSTM(hp.Int(f"lstm_{i}_units",min_value=10,max_value=300,step=10),activation='sigmoid',return_sequences=True))
        
    model4.add(LSTM((hp.Int("output_units",min_value=1,max_value=1,step=1)),activation='tanh',return_sequences=False))
    model4.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    return model4

In [799]:
Qatar_bivar=f"{int(time.time())}"

In [800]:
tuner =RandomSearch(
    build_model4,
    objective="loss",
    overwrite=False,
    max_trials=3,
    executions_per_trial=1,
    directory=Qatar_bivar)

In [801]:
tuner.search(x=x_train_Qatar, y=y_train_Qatar, epochs=40,batch_size=1,validation_data=(x_test_Qatar,y_test_Qatar))

Trial 3 Complete [00h 08m 34s]
loss: 0.03314037621021271

Best loss So Far: 0.0015808757161721587
Total elapsed time: 00h 17m 58s
INFO:tensorflow:Oracle triggered exit


In [803]:
print(tuner.get_best_hyperparameters()[0].values)

{'input_units': 90, 'n_layers': 2, 'lstm_0_units': 230, 'output_units': 1, 'lstm_1_units': 10}


In [804]:
print(tuner.results_summary())

Results summary
Results in 1639101713\untitled_project
Showing 10 best trials
Objective(name='loss', direction='min')
Trial summary
Hyperparameters:
input_units: 90
n_layers: 2
lstm_0_units: 230
output_units: 1
lstm_1_units: 10
Score: 0.0015808757161721587
Trial summary
Hyperparameters:
input_units: 10
n_layers: 1
lstm_0_units: 10
output_units: 1
Score: 0.0018930812366306782
Trial summary
Hyperparameters:
input_units: 90
n_layers: 4
lstm_0_units: 260
output_units: 1
lstm_1_units: 150
lstm_2_units: 10
lstm_3_units: 10
Score: 0.03314037621021271
None


In [805]:
print(tuner.get_best_models()[0].summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 90)             33480     
                                                                 
 lstm_1 (LSTM)               (None, 7, 230)            295320    
                                                                 
 lstm_2 (LSTM)               (None, 7, 10)             9640      
                                                                 
 lstm_3 (LSTM)               (None, 1)                 48        
                                                                 
Total params: 338,488
Trainable params: 338,488
Non-trainable params: 0
_________________________________________________________________
None


In [806]:
OmanCOVID=df[["Oman_new_cases","Oman_Mean_Temp"]]
OmanCOVID.shape

(513, 2)

In [807]:
scalarOmanCOVID=MinMaxScaler()
scalarOmanCOVID.fit(OmanCOVID)
normalizedOmanCOVID=scalarOmanCOVID.transform(OmanCOVID)

print(type(normalizedOmanCOVID[2][0]))
print(normalizedOmanCOVID[0][0],normalizedOmanCOVID[0][1])

<class 'numpy.float64'>
0.014108352144469524 0.41666666666666674


In [808]:
from numpy import hstack
n_input=7
n_output=1
casesOman=[]
Omantemp=[]

number=normalizedOmanCOVID.shape[0]
print(number)
print(type(normalizedOmanCOVID))
targetOman=[]
targetCases5=normalizedOmanCOVID[:,0]
for i in range(0,number):
    casesOman.append(normalizedOmanCOVID[i][0])
    Omantemp.append(normalizedOmanCOVID[i][1])
    

casesOman=np.array(casesOman)
Omantemp=np.array(Omantemp)

print(len(casesOman),len(Omantemp))
casesOman=casesOman.reshape(len(casesOman),1)
Omantemp=Omantemp.reshape(len(Omantemp),1)

print(casesOman.shape)
datasetOman=hstack((casesOman,Omantemp))
print(datasetOman.shape)
generatorOman=TimeseriesGenerator(datasetOman,targetCases5,length=n_input,batch_size=1)
print(generatorOman[1])

513
<class 'numpy.ndarray'>
513 513
(513, 1)
(513, 2)
(array([[[0.03132054, 0.41666667],
        [0.02426637, 0.45833333],
        [0.04063205, 0.54166667],
        [0.02765237, 0.58333333],
        [0.02990971, 0.58333333],
        [0.02878104, 0.58333333],
        [0.        , 0.625     ]]]), array([0.05332957]))


In [809]:
dataOman=[]
targetOman=[]
for i in range(len(generatorOman)):
    data6,target6=generatorOman[i]
    dataOman.append(data6[0,:,:])
    xx=float(target6)
    targetOman.append(xx)
dataOman=np.array(dataOman)
targetOman=np.array(targetOman)
print(type(dataOman))
print(dataOman.shape)
print(targetOman.shape)
x_train_Oman,x_test_Oman,y_train_Oman,y_test_Oman=train_test_split(dataOman,targetOman,test_size=0.2,random_state=4)
print(type(dataOman))
print(x_train_Oman.shape)

<class 'numpy.ndarray'>
(506, 7, 2)
(506,)
<class 'numpy.ndarray'>
(404, 7, 2)


In [810]:
def build_model5(hp):
    model5=keras.models.Sequential()
    model5.add(LSTM((hp.Int("input_units",min_value=10,max_value=300,step=10)),input_shape=(x_train_Oman.shape[1],x_train_Oman.shape[2]),activation='sigmoid',return_sequences=True))
    for i in range(hp.Int("n_layers",1,4)):
        model5.add(LSTM(hp.Int(f"lstm_{i}_units",min_value=10,max_value=300,step=10),activation='sigmoid',return_sequences=True))
        
    model5.add(LSTM((hp.Int("output_units",min_value=1,max_value=1,step=1)),activation='tanh',return_sequences=False))
    model5.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    return model5

In [811]:
Oman_bivar=f"{int(time.time())}"

In [812]:
tuner =RandomSearch(
    build_model5,
    objective="loss",
    overwrite=False,
    max_trials=3,
    executions_per_trial=1,
    directory=Oman_bivar)

In [813]:
tuner.search(x=x_train_Oman, y=y_train_Oman, epochs=40,batch_size=1,validation_data=(x_test_Oman,y_test_Oman))

Trial 3 Complete [00h 09m 01s]
loss: 0.012322972528636456

Best loss So Far: 0.007750484626740217
Total elapsed time: 00h 21m 29s
INFO:tensorflow:Oracle triggered exit


In [814]:
print(tuner.get_best_hyperparameters()[0].values)

{'input_units': 120, 'n_layers': 3, 'lstm_0_units': 120, 'output_units': 1, 'lstm_1_units': 160, 'lstm_2_units': 20}


In [815]:
print(tuner.results_summary())

Results summary
Results in 1639103014\untitled_project
Showing 10 best trials
Objective(name='loss', direction='min')
Trial summary
Hyperparameters:
input_units: 120
n_layers: 3
lstm_0_units: 120
output_units: 1
lstm_1_units: 160
lstm_2_units: 20
Score: 0.007750484626740217
Trial summary
Hyperparameters:
input_units: 10
n_layers: 3
lstm_0_units: 90
output_units: 1
lstm_1_units: 10
lstm_2_units: 10
Score: 0.009359140880405903
Trial summary
Hyperparameters:
input_units: 260
n_layers: 4
lstm_0_units: 100
output_units: 1
lstm_1_units: 230
lstm_2_units: 80
lstm_3_units: 10
Score: 0.012322972528636456
None


In [816]:
print(tuner.get_best_models()[0].summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 120)            59040     
                                                                 
 lstm_1 (LSTM)               (None, 7, 120)            115680    
                                                                 
 lstm_2 (LSTM)               (None, 7, 160)            179840    
                                                                 
 lstm_3 (LSTM)               (None, 7, 20)             14480     
                                                                 
 lstm_4 (LSTM)               (None, 1)                 88        
                                                                 
Total params: 369,128
Trainable params: 369,128
Non-trainable params: 0
_________________________________________________________________
None
